# Image Extraction and Annotation using PyDicom

In [ ]:

from AdjustBoundary import adjustContour
from helper import find_files, read_image, selectFrames, save_images
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pydicom
import os
import matplotlib
matplotlib.use('Qt5Agg')

image_files = find_files()
frNum = int(input('Enter frame number: '))
selectFrames(image_files, frNum-1, frNum)   # Show the selected frames of all the images
i = 0
crop = 30

while i < len(image_files):
    ctr = None
    name = image_files[i].split(".")[0]
    c_save = name + "_inx.txt"

    if os.path.isfile(c_save):           ctr = np.loadtxt(c_save)
    im = read_image(image_files[i], frNum)
    im = im[crop:-crop,crop:-crop]

    exit_mode = adjustContour(im, ctr, im.shape, crop, c_save)
    save_images(image_files[i], frNum)
    if exit_mode == 'exit':                 break
    if exit_mode == 'previous':             i = i - 1
    if exit_mode == 'next':                 i = i + 1
    if exit_mode == 'done':                 pass
    i += 1


# Thresholding the given Image

In [1]:
import cv2 

src = cv2.imread("save/IM_0013_1.jpg", cv2.IMREAD_GRAYSCALE); 
th, dst = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY); 
cv2.imwrite("opencv-threshold-example.jpg", dst); 

# # Thresholding with maxValue set to 128
#th, dst = cv2.threshold(src, 0, 128, cv2.THRESH_BINARY); 
#cv2.imwrite("opencv-thresh-binary-maxval.jpg", dst); 

# th, dst = cv2.threshold(src,127,255, cv2.THRESH_BINARY); 
# cv2.imwrite("opencv-thresh-binary.jpg", dst); 

# th, dst = cv2.threshold(src,127,255, cv2.THRESH_BINARY_INV); 
# cv2.imwrite("opencv-thresh-binary-inv.jpg", dst); 

# th, dst = cv2.threshold(src,127,255, cv2.THRESH_TRUNC); 
# cv2.imwrite("opencv-thresh-trunc.jpg", dst); 

# th, dst = cv2.threshold(src,127,255, cv2.THRESH_TOZERO); 
# cv2.imwrite("opencv-thresh-tozero.jpg", dst); 

# th, dst = cv2.threshold(src,127,255, cv2.THRESH_TOZERO_INV); 
# cv2.imwrite("opencv-thresh-to-zero-inv.jpg", dst); 


# Finding and Drawing the Contours based on thresholded Image

In [2]:
# detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
contours, hierarchy = cv2.findContours(image=dst, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
                                     
# draw contours on the original image
image_copy = dst.copy()
cv2.drawContours(image=image_copy, contours=contours[9], contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
               
# see the results
cv2.imshow('None approximation', image_copy)
cv2.waitKey(0)
cv2.imwrite('contours_none_image1.jpg', image_copy)
cv2.destroyAllWindows()

In [3]:
src = cv2.imread("opencv-threshold-example.jpg")
cv2.drawContours(image=src, contours=contours[9], contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
cv2.imshow('None approximation', src)
cv2.waitKey(0)
cv2.imwrite('contours_none_image1.jpg', src)
cv2.destroyAllWindows()

# Functions for automating the Process

In [4]:
import csv
def find_centroid(filepath):
    xes = []
    ys = []
    with open(filepath) as f:
         for line in f:
             x, y = map(float,line.split())
             xes.append(x)
             ys.append(y)
    x_cent=int(sum(xes)/len(xes))
    y_cent=int(sum(ys)/len(ys))
    #print(x_cent*3.14)
    #print(y_cent*3.22)
    return (int(x_cent*3.14),int(y_cent*3.22))

In [5]:
import operator
def cal_index(contours):
    area={}
    ind=[]
    c=[]
    for i,x in enumerate(contours):
        area[i]=cv2.contourArea(x)
    desc_area=sorted(area.items(),key=operator.itemgetter(1),reverse=True)
    de=desc_area[:6]
    for i in range(len(de)):
        ind.append(de[i][0])
    for i in ind:
        c.append(contours[i])
    print(ind)
    for x in range(len(ind)-1,-1,-1):
        if cv2.pointPolygonTest(contours[ind[x]], find_centroid('data/IM_0008-01_inx.txt'), False) == 1.0:
            print(x)
            return ind[x]
     

In [9]:
import cv2 
src = cv2.imread("save/IM_0008_1.jpg", cv2.IMREAD_GRAYSCALE); 
th, dst = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY); 
cv2.imwrite("opencv-threshold-example.jpg", dst);
contours, hierarchy = cv2.findContours(image=dst, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
contours1=contours
image_copy = cv2.imread("opencv-threshold-example.jpg")
cv2.drawContours(image=image_copy, contours=contours1[cal_index(contours)], contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
cv2.imshow('None approximation', image_copy)
cv2.waitKey(0)
cv2.imwrite('contours_none_image1.jpg', image_copy)
cv2.destroyAllWindows()

[1560, 2051, 279, 1806, 780, 1136]
1


In [10]:
def image_manipulation(src):
    cv2.imwrite("opencv-original-example.jpg",src)
    (th, dst) = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY); 
    cv2.imwrite("opencv-threshold-example.jpg", dst);
    contours, hierarchy = cv2.findContours(image=dst, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    image_copy = cv2.imread("opencv-original-example.jpg")
    cv2.drawContours(image=image_copy, contours=contours[cal_index(contours)], contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    cv2.imwrite('contours_none_image1.jpg', image_copy)
    return image_copy

In [12]:
import time
import cv2
cap=cv2.VideoCapture('mov-series-008.avi')
while(1):
    ret, frame = cap.read()  
    frame=cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    frame=image_manipulation(frame)
    cv2.imshow('Original', frame) 
    time.sleep(0.05)
    key=cv2.waitKey(1)
    if key==27:
        break
cap.release()
cv2.destroyAllWindows()

[173, 185, 164, 84, 172, 144]


TypeError: tuple indices must be integers or slices, not NoneType

# Method:Mean Filter

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from PIL import Image, ImageFilter
%matplotlib inline
image = cv2.imread('opencv-threshold-example.jpg') # reads the image
image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # convert to HSV
figure_size = 9 # the dimension of the x and y axis of the kernal.
new_image = cv2.blur(image,(figure_size, figure_size))
plt.figure(figsize=(11,6))
plt.subplot(121), plt.imshow(cv2.cvtColor(image, cv2.COLOR_HSV2RGB)),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_HSV2RGB)),plt.title('Mean filter')
plt.xticks([]), plt.yticks([])
plt.show()


In [ ]:
import cv2 

src = cv2.imread("save/IM_0008_1.jpg"); 
src = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
src = cv2.blur(src,(2,2))
src=cv2.cvtColor(src, cv2.COLOR_HSV2RGB)
src=cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
th, dst = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY);

cv2.imwrite("opencv-threshold-example.jpg", dst);
contours, hierarchy = cv2.findContours(image=dst, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
image_copy = cv2.imread("opencv-threshold-example.jpg")
cv2.drawContours(image=image_copy, contours=contours[156], contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
cv2.imshow('None approximation', image_copy)
cv2.waitKey(0)
cv2.imwrite('contours_none_image1.jpg', image_copy)
cv2.destroyAllWindows()

# Centroid Calculation


In [ ]:
import csv
def find_centroid(filepath):
    xes = []
    ys = []
    with open(filepath) as f:
         for line in f:
             x, y = map(float,line.split())
             xes.append(x)
             ys.append(y)
    x_cent=int(sum(xes)/len(xes))
    y_cent=int(sum(ys)/len(ys))
    #print(x_cent*3.14)
    #print(y_cent*3.22)
    return (int(x_cent),int(y_cent))


In [ ]:
img=cv2.imread('save/IM_0008_1.jpg')
img = cv2.circle(img, find_centroid('data/IM_0008-01_inx.txt'), 5, (0,255,0), 10)
cv2.imshow('Nosdfs', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
  
# function to display the coordinates of
# of the points clicked on the image
def click_event(event, x, y, flags, params):
 
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', img)
 
    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(img, str(b) + ',' +
                    str(g) + ',' + str(r),
                    (x,y), font, 1,
                    (255, 255, 0), 2)
        cv2.imshow('image', img)
 
# driver function
if __name__=="__main__":
 
    # reading the image
    img = cv2.imread('save/IM_0003_1.jpg', 1)
 
    # displaying the image
    cv2.imshow('image', img)
 
    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback('image', click_event)
 
    # wait for a key to be pressed to exit
    cv2.waitKey(0)
 
    # close the window
    cv2.destroyAllWindows()

In [ ]:
def centroid(c):
    xes = []
    ys = []
    for line in c:
        x, y = map(float,line.split())
        xes.append(x)
        ys.append(y)
    x_cent=int(sum(xes)/len(xes))
    y_cent=int(sum(ys)/len(ys))
    return (int(x_cent*3.14),int(y_cent*3.22))


In [ ]:
def near_centroid(centroid,contours):
    a=[x for x in range(0,len(contours))]
    diff=100000000
    d=0
    for i,x in zip(a,contours):
        odd_i = []
        even_i = []
        st=x.reshape(-1)
        for i in range(0, len(st)):
            if i % 2:
                even_i.append(st[i])
            else :
                odd_i.append(x.reshape(-1)[i])
        xcord=sum(odd_i)/len(odd_i)    
        ycord=sum(even_i)/len(even_i)
        total=abs((xcord*3.12)-centroid[0])+abs((ycord*3.22)-centroid[1])
        #print(total)
        if total<diff:
            diff=total
            d=i
    return d
        
        

In [ ]:
a=[x for x in range(0,len(contours))]
for i,cnt in zip(a,contours):
    print(contours,i)
    for x,y in cnt[i]:
        print(x,y)
    

In [ ]:
print(near_centroid((418,248),contours))

In [ ]:
contours[0].reshape(-1)

In [ ]:
contours[1].reshape(-1)

In [ ]:
odd_i = []
even_i = []
for i in range(0, len(contours[1].reshape(-1))):
    if i % 2:
        even_i.append(contours[1].reshape(-1)[i])
    else :
        odd_i.append(contours[1].reshape(-1)[i])
odd_i

In [ ]:
sum(even_i)/len(even_i)

In [ ]:
even_i

In [ ]:
import numpy as np
def cal_centroid(centroid,contours):
    diff1=0
    diff2=0
    c=0
    d=0
    a=[x for x in range(0,len(contours))]
    for i,x in zip(a,contours):
        M = cv2.moments(x)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        tot= np.sqrt(abs((cx-centroid[0])**2+(cy-centroid[1]**2)))
        if tot>diff1:
            diff1= tot
            c=i
    for i,x in zip(a,contours):
        M = cv2.moments(x)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        tot= np.sqrt(abs((cx-centroid[0])**2+(cy-centroid[1]**2)))
        if tot>diff2 and tot!=diff1:
            diff2= tot
            d=i
    return d

In [ ]:
cal_centroid((124,102),c)

In [ ]:
ind

In [ ]:
print(cv2.moments(c[0]))

In [ ]:
result = cv2.pointPolygonTest(contours[1560], (418,248), False) 
result

In [ ]:
contours[2051]